# Who owns Charlottesville?
> "Mapping the city's biggest property owners"

- author: Josh Carp
- categories: [land-use]

Every afternoon, [@everysalecville](https://twitter.com/everysalecville) posts details of the real estate transactions of the past day. Often, the buyer is a Limited Liability Company, or LLC; recent buyers include "YELLOW SHIRT LLC", "DADDY RABBIT, INC", and "ASPIRING DEVELOPMENTS, LLC". Which of these companies own the most land in Charlottesville, and who owns the companies?

We can look up real estate tranactions from the city [Open Data Portal](https://opendata.charlottesville.org/datasets/real-estate-sales) and identify the top property owners by owner name or address. In some cases, multiple owner names share the same owner address—often a business address or Post Office box—so we group properties by owner address. For example, 148 properties list their owner address as "224 14TH ST NW", with owner names like "400 PRESTON AVENUE LLC", "DOGWOOD PROPERTIES OF C'VILLE LLC", and "WP MONTEBELLO, LLC". These owning entities all seem to be associated with [Woodard Properties](https://woodardproperties.com/), so we label them as part of this entity.

Note: I'm excluding tax-exempt parcels from this post, which is why we don't see much property owned by the city or UVA here.

In [1]:
#collapse-hide
import datetime

import branca
import folium
import palettable
import pandas as pd
from google.cloud import bigquery

bq = bigquery.Client()

pd.set_option("display.max_colwidth", None)

/Users/josh.carp/miniconda3/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
#collapse-hide
holdings = bq.query(
    """
    with parcels as (
      select
        parcelnumb as parcelnumber,
        assessment,
        ownername,
        case
          when owneraddre = 'MSC BOX 5186' then 'P O BOX 5306'
          when owneraddre = '3056 BERKMAR DRIVE' then '3056 BERKMAR DR'
          else owneraddre
        end as owneraddress,
        st_geogfromgeojson(geometry) as geometry,
      from `cvilledata.cville_open_data.parcel_area_details`
      where owneraddre is not null
        and filetype not in ('E')
    )
    select
      owneraddress,
      array_agg(distinct ownername ignore nulls order by ownername) as ownernames,
      sum(assessment) as assessment,
      st_area(st_union_agg(geometry)) as area,
      count(*) as count,
    from parcels
    group by owneraddress
    order by count desc
    """
).result().to_dataframe()

In [3]:
#collapse-hide
parcels = bq.query(
    """
    with parcels as (
      select
        parcelnumb as parcelnumber,
        streetnumb as streetnumber,
        streetname,
        concat(coalesce(streetnumb, '?'), ' ', streetname) as address,
        assessment,
        ownername,
        case
          when owneraddre = 'MSC BOX 5186' then 'P O BOX 5306'
          when owneraddre = '3056 BERKMAR DRIVE' then '3056 BERKMAR DR'
          else owneraddre
        end as owneraddress,
        st_geogfromgeojson(geometry) as geometry,
      from `cvilledata.cville_open_data.parcel_area_details`
      where owneraddre is not null
        and filetype not in ('E')
    ), lastsold as (
      select
        parcelnumb as parcelnumber,
        max(saledate) as saledate,
      from `cvilledata.cville_open_data.real_estate_sales`
      where saleamount > 0
      group by parcelnumb
    )
    select
      parcels.*,
      lastsold.saledate as lastsold,
    from parcels
    left join lastsold on parcels.parcelnumber = lastsold.parcelnumber
    where owneraddress in unnest(@owneraddresses)
    """,
    job_config=bigquery.QueryJobConfig(
        query_parameters=[bigquery.ArrayQueryParameter("owneraddresses", "STRING", holdings[:10].owneraddress.tolist())]
    ),
).result().to_geodataframe()

In [4]:
#collapse-hide
ADDRESS_TO_ENTITY_NAME = {
    "810 CATALPA CT": "Neighborhood Investments",
    "224 14TH ST NW": "Woodard Properties",
    "PO BOX 911": "City of Charlottesville",
    "360 ARDWOOD RD": "Silk Purse Properties",
    "P O BOX 400884": "Rector & Visitors of UVA",
    "977 SEMINOLE TR STE 329": "Evening Properties",
    "PO BOX 1405": "CRHA",
    "P O BOX 1414": "CBS Rentals",
    "P O BOX 5306": "MSC UVA",
    "P O BOX 1467": "Riverbend Development",
    "1134 E HIGH ST": "Alcova Properties",
    "1500 AMHERST ST #3": "Real Property Management",
    "142 S PANTOPS DR": "Southern Development Homes",
    "3056 BERKMAR DR": "Wade Rentals",
    "1001 E MARKET ST STE 102": "Community Services Housing",
    "400 LOCUST AVE STE 3": "BMC Holdings Group",
    "300 2ND ST NE": "Milestone Partners",
    "2000 BENTIVAR DR": "Binx Properties",
    "2088 UNION ST STE 1": "Allan Cadgene",
    "P O BOX 5526": "Great Eastern Management Company",
    "201 15TH ST NW STE 1A": "Veliky Rentals",
    "P O BOX 400218": "University of Virginia Foundation",
    "P O BOX 7136": "INV Group",
    "P O BOX 9035": "Pepsi-Cola Central Virginia",
}

holdings["ownerentity"] = holdings.owneraddress.apply(lambda owneraddress: ADDRESS_TO_ENTITY_NAME.get(owneraddress, owneraddress))
parcels["ownerentity"] = parcels.owneraddress.apply(lambda owneraddress: ADDRESS_TO_ENTITY_NAME.get(owneraddress, owneraddress))

parcels["lastsold"] = parcels.lastsold.apply(lambda date: datetime.datetime.strftime(date, "%Y-%m-%d") if date else "")


Here's the complete list of owner names associated with Woodard Properties via its business address:

In [5]:
#collapse-hide
holdings.ownernames[:1].tolist()

[array(['400 PRESTON AVENUE LLC', 'C-VILLE BUSINESS PARK LLC',
        'C-VILLE BUSINESS PARK, LLC', 'CAMDEN PLAZA, LLC',
        'CHARLOTTESVILLE FIRST STREET APTS LLC', 'COMYN GROUNDS, LLC',
        "DOGWOOD PROPERTIES OF C'VILLE LLC",
        "DOGWOOD PROPERTIES OF C'VILLE, LLC", 'EAST MARKET LLC',
        'FIRST AND MAIN CHARLOTTESVILLE, LLC',
        'GOODMAN STREET PROPERTIES LLC', 'GRADUATE APARTMENTS, LLC',
        'GRADUATE COURT I LLC', 'GRADUATE PLACE LLC',
        'GRADY APARTMENTS LLC', 'KOW DEVELOPMENT LLC', 'MARKET PLAZA LLC',
        'MARKET SQUARE LLC', "MOORE'S CREEK LLC", 'PINK WAREHOUSE, LLC',
        'RENAISSANCE PLACE LLC', 'THE CORNER PLACE LLC',
        'WERTLAND TRIO LLC', 'WEST SOUTH STREET LLC', 'WOODARD UNITED LLC',
        'WP CASC LLC', 'WP CHERRY LLC', 'WP FACILITIES LLC',
        'WP FOREST STREET LLC', 'WP GROUP LLC', 'WP MONTEBELLO, LLC',
        'WP PRESTON LLC', 'WP SOUTH STREET LLC', 'WP UNIVERSITY LLC'],
       dtype=object)]

And here are the top 20 property owners in the city, ranked by number of parcels owned. Some business entities, like Silk Purse Properties or Evening Properties, use a single LLC to manage all their properties. Others use a range of LLCs, some named by property like "114 CARROLLTON TERRACE, LLC" and others with less revealing names like "MISCELLANEOUS PROPERTIES, LLC".

Note: owner names and addresses are public records from the city open data portal, but owner entities are my best guess at the person or corporate entity that owns the properties. I looked up owner entities by Googling owner and property addresses and cross-checking property holdings with corporate rental portals. I've tried to be thorough but may have made mistakes, and I haven't been able to track down the owner of Wertland Commons, which appears to be registered at a residence.

In [6]:
#collapse-hide
holdings[["owneraddress", "ownernames", "ownerentity", "count"]][:20]

,owneraddress,ownernames,ownerentity,count
0,224 14TH ST NW,"[400 PRESTON AVENUE LLC, C-VILLE BUSINESS PARK LLC, C-VILLE BUSINESS PARK, LLC, CAMDEN PLAZA, LLC, CHARLOTTESVILLE FIRST STREET APTS LLC, COMYN GROUNDS, LLC, DOGWOOD PROPERTIES OF C'VILLE LLC, DOGWOOD PROPERTIES OF C'VILLE, LLC, EAST MARKET LLC, FIRST AND MAIN CHARLOTTESVILLE, LLC, GOODMAN STREET PROPERTIES LLC, GRADUATE APARTMENTS, LLC, GRADUATE COURT I LLC, GRADUATE PLACE LLC, GRADY APARTMENTS LLC, KOW DEVELOPMENT LLC, MARKET PLAZA LLC, MARKET SQUARE LLC, MOORE'S CREEK LLC, PINK WAREHOUSE, LLC, RENAISSANCE PLACE LLC, THE CORNER PLACE LLC, WERTLAND TRIO LLC, WEST SOUTH STREET LLC, WOODARD UNITED LLC, WP CASC LLC, WP CHERRY LLC, WP FACILITIES LLC, WP FOREST STREET LLC, WP GROUP LLC, WP MONTEBELLO, LLC, WP PRESTON LLC, WP SOUTH STREET LLC, WP UNIVERSITY LLC]",Woodard Properties,148
1,810 CATALPA CT,"[NEIGHBORHOOD INVESTMENTS LLC, NEIGHBORHOOD INVESTMENTS, LLC, NEIGHBORHOOD INVESTMENTS--1725, LLC, NEIGHBORHOOD INVESTMENTS--1910, LLC, NEIGHBORHOOD INVESTMENTS--GRC, LLC, NEIGHBORHOOD INVESTMENTS--JS, LLC, NEIGHBORHOOD INVESTMENTS--LL, LLC, NEIGHBORHOOD INVESTMENTS--PC, LP, NEIGHBORHOOD INVESTMENTS-1910, LLC, NEIGHBORHOOD INVESTMENTS-324, LLC, NEIGHBORHOOD INVESTMENTS-CA, LLC, NEIGHBORHOOD INVESTMENTS-JJ, LLC, NEIGHBORHOOD INVESTMENTS-RC, LLC, NEIGHBORHOOD INVESTMENTS-RH, LLC, NEIGHBORHOOD INVESTMENTS-WS, LLC, SANDBOX, LLC, TARLETON SQUARE LAND, LLC]",Neighborhood Investments,71
2,3056 BERKMAR DR,"[JOHNSON VILLAGE, LLC, MISCELLANEOUS PROPERTIES, LLC, WELK PLACE LLC, WELK PLACE, LLC]",Wade Rentals,55
3,360 ARDWOOD RD,"[SILK PURSE PROPERTIES, LLC]",Silk Purse Properties,55
4,977 SEMINOLE TR STE 329,"[EVENING PROPERTIES, LC]",Evening Properties,49
5,P O BOX 1414,"[100 AVON LLC, BARON LLC, BARON, LLC, BETA BRIDGE LLC, BLUE COTTAGE LLC, BLUE COTTAGE, LLC, DINSMORE LLC, DINSMORE, L L C, DINSMORE, LLC, LITTLE HOUSE, LLC, MCKENNIE LLC, PIEDMONT HOSPITAL LLC, PIEDMONT HOSPITAL, LLC, STULTZ, LLC, THE GREENHOUSE APARTMENTS LLC, WERTLAND WAREHOUSE, LLC, WERTLAND, LLC]",CBS Rentals,49
6,1134 E HIGH ST,"[PARK LANE PROPERTIES, LLC, RIVANNA, PARTNERSHIP, UNIVERSITY LIMITED PARTNERSHIP]",Alcova Properties,47
7,P O BOX 5306,"[111 HARMON, LLC, 114 CARROLLTON TERRACE, LLC, 1337 PRESTON, LLC, 1701 GORDON AVENUE, LLC, 600 RUGBY ROAD HOUSING CORPORATION, BROOKWOOD PROPERTY OWNERS ASSOCIATION, INC, BURNET COMMONS PROP OWNER ASSOC, BURNET COMMONS TWO PROPERTY OWNERS ASSOCIATION, INC, CABELL LIMITED PARTNERSHIP, CAMBRIDGE HOUSE APARTMENTS, LLC, CAMBRIDGE SQUARE APARTMENTS LP, CATON, DOUGLAS E, CLUB MAD, LLC, FRED APARTMENTS, LLC, THE, HILLSIDE LIMITED PARTNERSHIP, L-R INVESTMENTS, LEWIS MOUNTAIN HOLDING, LLC, MOUNTAIN VIEW MOBILE HOME PARK, MOUNTAIN VIEW MOBILE HOME PARK LLC, OXFORD HILL LLC, PAYNE'S MILL PROPERTY OWNERS ASSOCIATION, INC, ROCK CREEK OWNERS ASSOCIATION INC, RUGBY MCINTYRE APARTMENTS, LLC, SADLER COURT APARTMENTS LLC, STADIUM ROAD LIMITED PARTNERSHIP, WELLINGTON COURT, LLC, WERTLAND STREET LLC, WILLOUGHBY TOWNES OWNERS ASSOC, INC, WOODROW TOO, LLC, WOODROW, LLC]",MSC UVA,45
8,P O BOX 1467,"[1023 PARK STREET, LLC, 1134 EMMET STREET, LLC, 1215 EAST MARKET STREET LLC, 321-323 EAST MAIN STREET, LLC, 700 PRESTON LLC, 901 SEMINOLE TRAIL, LLC, BELMONT & CARLTON HOLDINGS, LLC, BELMONT AND CARLTON HOLDINGS, LLC, CHOCO-CRUZ, LLC, CLEVELAND AVENUE, LLC, FLUVANNA HOLDINGS, LLC, JEFFERSON THEATER HOLDINGS, LLC, MATTIE, JEFFREY D, PEYTON ASSOCIATES PARTNERSHIP, PEYTON ASSOCIATES PARTNERSHIP, TRIMONT, LLC, WATER MAIN, LLC]",Riverbend Development,40
9,1001 E MARKET ST STE 102,"[CARLTON MANOR HOUSING, INC, CARLTON NEIGHBORHOOD HOUSING LLC, CH MEWS HOUSING, LP, COMMUNITY SERVICES HOUSING INC, COMMUNITY SERVICES HOUSING, INC, GRADY MANOR HOUSING, INC, MONTICELLO MANOR HOUSING, INC, SHORT 18TH STREET HOUSING, LP, SPRUCE MANOR HOUSING, INC]",Community Services Housing,34


In [7]:
#collapse-hide
# Adapted from https://github.com/mrcagney/examples_folium/blob/develop/notebooks/categorical_legend.ipynb
def add_categorical_legend(map_, title, colors, labels):
    """
    Given a Folium map, add to it a categorical legend with the given title, colors, and corresponding labels.
    The given colors and labels will be listed in the legend from top to bottom.
    Return the resulting map.
    
    Based on `this example <http://nbviewer.jupyter.org/gist/talbertc-usgs/18f8901fc98f109f2b71156cf3ac81cd>`_.
    """
    # Error check
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_by_label = dict(zip(labels, colors))

    # Make legend HTML
    template = f"""
    {{% macro html(this, kwargs) %}}

    <!doctype html>
    <html lang="en">
    <head>
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
    <div id='maplegend' class='maplegend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
    """

    for label, color in color_by_label.items():
        template += f"<li><span style='background:{color}'></span>{label}</li>"

    template += """
        </ul>
      </div>
    </div>

    </body>
    </html>

    <style type='text/css'>
      .maplegend {
        position: absolute;
        z-index:9999;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        right: 10px;
        bottom: 20px;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    {% endmacro %}
    """

    macro = branca.element.MacroElement()
    macro._template = branca.element.Template(template)
    map_.get_root().add_child(macro)

    return map_

Next, we can map the properties of the largest groups. Here's an interactive map of the top 10 property owners in the city; hover over a parcel to see its owner, current assessment, etc. Some groups specialize in student apartments, and most of their properties are near UVA. [MSC](https://livewithmsc.com/) mostly follows this pattern, but it also appears to own the Mountain View Mobile Home Park off Avon St. [Neighborhood Investments](https://www.neighborhoodprops.com/) also owns some student apartments near campus, as well as a number of townhomes on Longwood Dr. Silk Road Properties, owned by Jeremy Caplin, owns dozens of rentals in the 10th and Page neighborhood, which are ["offered them as affordable rentals for working families, while trying to slow displacement and gentrification"](https://www.linkedin.com/in/jeremy-caplin-65b1172a/). And [Woodard Properties](https://woodardproperties.com/), the largest property owner by number of parcels, owns properties all over the city, including a growing collection in Fifeville that now includes much of the land between King St and Elm St.

In [8]:
#collapse-hide
entities = holdings[:10].ownerentity.tolist()
palette = palettable.colorbrewer.qualitative.Set3_10

def style(shape):
    ownerentity = shape["properties"]["ownerentity"]
    fill_color = palette.hex_colors[entities.index(ownerentity)]
    return {
        "color": "black",
        "weight": 0.5,
        "fillColor": fill_color,
        "fillOpacity": 1,
    }

map_ = folium.Map(
    location=[38.04, -78.49],
    tiles="cartodbpositron",
    zoom_start=13.55,
    zoom_delta=0.5,
)
folium.GeoJson(
    parcels[["ownerentity", "ownername", "assessment", "address", "lastsold", "geometry"]],
    tooltip=folium.GeoJsonTooltip(fields=["ownerentity", "ownername", "assessment", "address", "lastsold"], labels=True),
    style_function=style,
).add_to(map_)

add_categorical_legend(map_, "Owners", palette.hex_colors, entities)

map_

Finally, we can check which entities have bought the most land in the city in recent years. The top buyer by parcel count is Stanley Martin Homes, which is tied to a [stalled development](https://www.cvilletomorrow.org/articles/neighbors-push-back-on-proposed-frys-spring-subdivision/) off Monte Vista Ave. Next are Southern Development Homes, Woodard Properties, and Neighborhood Investments, all familiar from our list of top property owners above. There are also a few owner entities that I haven't looked up, like "620 WOODBROOK DR STE 6" and "200 GARRETT ST STE O". I'll leave that as an exercise for the hypothetical reader.

In [9]:
#collapse-hide
sales = bq.query(
    """
    with parcels as (
      select
        parcelnumb as parcelnumber,
        assessment,
        ownername,
        case
          when owneraddre = 'MSC BOX 5186' then 'P O BOX 5306'
          when owneraddre = '3056 BERKMAR DRIVE' then '3056 BERKMAR DR'
          else owneraddre
        end as owneraddress,
        st_geogfromgeojson(geometry) as geometry,
      from `cvilledata.cville_open_data.parcel_area_details`
      where owneraddre is not null
        and filetype not in ('E')
    )
    select
      owneraddress,
      array_agg(distinct ownername ignore nulls order by ownername) as ownernames,
      sum(assessment) as assessment,
      st_area(st_union_agg(parcels.geometry)) as area,
      count(*) as count,
    from parcels
    join `cvilledata.cville_open_data.real_estate_sales` sales
      on parcels.parcelnumber = sales.parcelnumb
      and sales.saledate >= '2017-01-01' and sales.saledate < '2022-01-01'
      and sales.saleamount > 0
    group by owneraddress
    order by assessment desc
    """
).result().to_dataframe()

In [10]:
#collapse-hide
sales["ownerentity"] = sales.owneraddress.apply(lambda owneraddress: ADDRESS_TO_ENTITY_NAME.get(owneraddress, owneraddress))
sales[["owneraddress", "ownernames", "ownerentity", "count"]].sort_values("count", ascending=False)[:10]

,owneraddress,ownernames,ownerentity,count
132,11710 PLAZA AMERICA DR # 1100,"[STANLEY MARTIN HOMES, LLC]",11710 PLAZA AMERICA DR # 1100,36
27,142 S PANTOPS DR,"[BELMONT STATION, LLC, FMC INVESTMENTS, LLC, SOUTHERN PROPERTY, LLC]",Southern Development Homes,28
12,224 14TH ST NW,"[DOGWOOD PROPERTIES OF C'VILLE LLC, KOW DEVELOPMENT LLC, PINK WAREHOUSE, LLC, WEST SOUTH STREET LLC, WOODARD UNITED LLC, WP CASC LLC, WP CHERRY LLC, WP FACILITIES LLC, WP MONTEBELLO, LLC, WP PRESTON LLC, WP UNIVERSITY LLC]",Woodard Properties,26
11,810 CATALPA CT,"[NEIGHBORHOOD INVESTMENTS LLC, NEIGHBORHOOD INVESTMENTS, LLC, NEIGHBORHOOD INVESTMENTS--1910, LLC, NEIGHBORHOOD INVESTMENTS--GRC, LLC, NEIGHBORHOOD INVESTMENTS--JS, LLC, NEIGHBORHOOD INVESTMENTS-1910, LLC]",Neighborhood Investments,23
16,400 LOCUST AVE STE 3,"[600 CONCORD LLC, 618 FOREST LLC, 9.5 STREET LLC, 910 EAST HIGH LLC, EAST JEFFERSON ASSEMBLAGE LLC, SUNRISE CVILLE LLC, TIN HUT LLC]",BMC Holdings Group,14
195,620 WOODBROOK DR STE 6,"[132 CARLTON RD, LLC, ACCESSO PROPERTIES, LLC, CROSBY, JAMES & CATHERINE, HNS GROUP, LLC, TOWERS 209, LLC]",620 WOODBROOK DR STE 6,12
6,200 GARRETT ST STE O,"[632 PARK STREET, LLC, DAIRY CENTRAL PHASE 1, LLC, DAIRY HOLDINGS, LLC, GRI REALTY HOLDINGS, LLC, PRESTON PLACE, LLC]",200 GARRETT ST STE O,12
7,10120 W BROAD ST STE J,"[QUIRK CHARLOTTESVILLE, LLC, TOW LOT LLC]",10120 W BROAD ST STE J,11
10,P O BOX 400218,"[UNIVERSITY OF VIRGINIA FOUNDATION, UNIVERSITY OF VIRGINIA FOUNDATION]",University of Virginia Foundation,10
15,P O BOX 5526,"[SEQUEL INVESTORS LIMITED PARTNERSHIP, THE RESIDENCES AT 218, LLC]",Great Eastern Management Company,9


This is just a quick analysis, but hopefully it's been interesting. You can send corrections or suggestions to me at [@whatthecarp](https://twitter.com/whatthecarp), or do your own analysis and let me know about it. You can find source data from the city [Open Data Portal](https://opendata.charlottesville.org/), and if you enjoy SQL, I've mirrored the relevant tables to a [public dataset](https://console.cloud.google.com/bigquery?p=cvilledata&d=cville_open_data&page=dataset) on Google BigQuery.